In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)


In [2]:
df = pd.read_csv("../data/processed/moneypuck_modern.csv")
df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,...,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2017,NYR,2017020007,NYR,COL,HOME,20171005,Team Level,all,0.5110,0.6106,0.6067,3600.0,38.414,3.840,2.957,7.426,1.146,22.469,16.163,3.730,3.728,3.619,39.0,...,16.0,20.0,21.0,6.0,7.0,22.0,8.0,5.0,0.631,1.077,1.966,1.0,1.0,2.0,46.423,35.0,36.872,5.0,0.455,0.196,0.502,3.627,3.748,3.729,0
1,NYR,2017,NYR,2017020016,NYR,TOR,AWAY,20171007,Team Level,all,0.4192,0.4275,0.4623,3600.0,35.487,3.137,2.027,8.499,1.091,19.656,14.590,3.085,3.158,3.106,35.0,...,4.0,46.0,21.0,17.0,13.0,34.0,20.0,3.0,1.248,2.481,0.616,1.0,6.0,1.0,77.372,57.0,57.805,8.0,0.626,0.627,0.627,4.345,4.350,4.234,0
2,NYR,2017,NYR,2017020031,NYR,MTL,HOME,20171008,Team Level,all,0.4285,0.4057,0.4250,3600.0,24.414,2.264,1.915,4.780,0.732,14.439,9.870,2.199,2.224,2.161,25.0,...,6.0,22.0,22.0,6.0,12.0,38.0,4.0,4.0,1.217,0.486,1.317,0.0,0.0,0.0,62.332,46.0,45.525,6.0,0.491,0.792,0.792,2.718,2.752,2.649,0
3,NYR,2017,NYR,2017020039,NYR,STL,HOME,20171010,Team Level,all,0.4662,0.6000,0.5733,3600.0,31.600,2.563,2.295,8.037,1.041,17.283,11.781,2.462,2.439,2.346,33.0,...,8.0,26.0,24.0,6.0,9.0,25.0,3.0,4.0,0.493,0.325,2.117,1.0,1.0,1.0,44.696,32.0,34.100,6.0,0.341,0.312,0.312,2.964,3.052,3.041,0
4,NYR,2017,NYR,2017020059,NYR,CBJ,AWAY,20171013,Team Level,all,0.2938,0.4444,0.4343,3600.0,30.837,2.131,2.097,6.821,0.954,17.386,13.612,2.079,2.256,2.201,38.0,...,10.0,33.0,11.0,11.0,5.0,37.0,13.0,6.0,1.196,1.612,2.314,1.0,1.0,1.0,67.245,56.0,54.098,1.0,0.594,1.341,1.341,4.376,4.235,4.083,0


In [6]:
df["gameDate"] = pd.to_datetime(df["gameDate"], format="%Y%m%d")

In [7]:
df = df.sort_values(["team", "gameDate"]).reset_index(drop=True)

In [8]:
df["days_rest"] = df.groupby("team")["gameDate"].diff().dt.days - 1

In [9]:
df["days_rest"] = df["days_rest"].fillna(3)

In [10]:
df["back_to_back"] = (df["days_rest"] == 0).astype(int)

In [11]:
df[["team", "gameDate", "days_rest", "back_to_back"]].head(10)

,team,gameDate,days_rest,back_to_back
0,ANA,2017-10-05,3.0,0
1,ANA,2017-10-07,1.0,0
2,ANA,2017-10-09,1.0,0
3,ANA,2017-10-11,1.0,0
4,ANA,2017-10-13,1.0,0
5,ANA,2017-10-15,1.0,0
6,ANA,2017-10-20,4.0,0
7,ANA,2017-10-24,3.0,0
8,ANA,2017-10-26,1.0,0
9,ANA,2017-10-28,1.0,0


In [12]:
df["days_rest"].value_counts().sort_index()

days_rest
0.0       3270
1.0      12628
2.0       3930
3.0       1004
4.0        365
         ...  
182.0        2
183.0        1
308.0        3
309.0        3
310.0        1
Name: count, Length: 91, dtype: int64

In [13]:
df.to_csv("../data/processed/moneypuck_features_step1.csv", index=False)

In [15]:
df["rolling_goals_5"] = (
    df.groupby("team")["goalsFor"]
      .shift(1)
      .rolling(window=5)
      .mean()
)

In [16]:
df["rolling_xg_5"] = (
    df.groupby("team")["xGoalsFor"]
      .shift(1)
      .rolling(window=5)
      .mean()
)

In [18]:
df["rolling_goals_5"] = (
    df.groupby("team")["rolling_goals_5"]
      .transform(lambda x: x.fillna(x.expanding().mean()))
)

df["rolling_xg_5"] = (
    df.groupby("team")["rolling_xg_5"]
      .transform(lambda x: x.fillna(x.expanding().mean()))
)

In [19]:
df[["team", "gameDate", "goalsFor", "rolling_goals_5", "rolling_xg_5"]].head(10)

,team,gameDate,goalsFor,rolling_goals_5,rolling_xg_5
0,ANA,2017-10-05,5.0,NaN,NaN
1,ANA,2017-10-07,2.0,NaN,NaN
2,ANA,2017-10-09,0.0,NaN,NaN
3,ANA,2017-10-11,3.0,NaN,NaN
4,ANA,2017-10-13,1.0,NaN,NaN
5,ANA,2017-10-15,1.0,2.2,2.4904
6,ANA,2017-10-20,6.0,1.4,2.1482
7,ANA,2017-10-24,6.0,2.2,2.3950
8,ANA,2017-10-26,3.0,3.4,2.1622
9,ANA,2017-10-28,4.0,3.4,2.1410


In [20]:
df.to_csv("../data/processed/moneypuck_features_step2.csv", index=False)

In [21]:
# Define simple fatigue penalty factors
df["fatigue_penalty"] = 1 - 0.1 * df["back_to_back"]

In [22]:
df["fatigue_adj_goals"] = df["rolling_goals_5"] * df["fatigue_penalty"]

In [23]:
df["fatigue_adj_xg"] = df["rolling_xg_5"] * df["fatigue_penalty"]

In [24]:
df[
    ["team", "gameDate", "back_to_back", "rolling_goals_5", "fatigue_adj_goals"]
].head(12)

,team,gameDate,back_to_back,rolling_goals_5,fatigue_adj_goals
0,ANA,2017-10-05,0,NaN,NaN
1,ANA,2017-10-07,0,NaN,NaN
2,ANA,2017-10-09,0,NaN,NaN
3,ANA,2017-10-11,0,NaN,NaN
4,ANA,2017-10-13,0,NaN,NaN
5,ANA,2017-10-15,0,2.2,2.2
6,ANA,2017-10-20,0,1.4,1.4
7,ANA,2017-10-24,0,2.2,2.2
8,ANA,2017-10-26,0,3.4,3.4
9,ANA,2017-10-28,0,3.4,3.4


In [25]:
df.to_csv("../data/processed/moneypuck_features_final.csv", index=False)